<a href="https://colab.research.google.com/github/Andreylive/UAV-human-detection/blob/main/Faster_R_CNN_for_UAV_human_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install detectron 2
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
# Check installed libruaries
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
# Install additonal libruaries and dataset

# Common libruaries
import os
import cv2

from datetime import datetime
from google.colab.patches import cv2_imshow

# Dataset preparationand loading
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog

# Vizualization
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode

# Configuration
from detectron2 import model_zoo
from detectron2.config import get_cfg

# Evaluation
from detectron2.engine import DefaultPredictor

# Trainning
from detectron2.engine import DefaultTrainer

In [ ]:
# Download HERIAL dataset
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="HD0OPLHEvjhSna3fqMZq")
project = rf.workspace("new-workspace-qsuag").project("heridal-t428z")
dataset = project.version(2).download("coco")

In [ ]:
# Register model
DATA_SET_NAME = dataset.name.replace("-", "_")+'2'
ANNOTATIONS_FILE_NAME = "_annotations.coco.json"

TRAIN_DATA_SET_NAME = f"{DATA_SET_NAME}-train"
TRAIN_DATA_SET_IMAGES_DIR_PATH = os.path.join(dataset.location, "train")
TRAIN_DATA_SET_ANN_FILE_PATH = os.path.join(dataset.location, "train", ANNOTATIONS_FILE_NAME)

register_coco_instances(
    name=TRAIN_DATA_SET_NAME, 
    metadata={}, 
    json_file=TRAIN_DATA_SET_ANN_FILE_PATH, 
    image_root=TRAIN_DATA_SET_IMAGES_DIR_PATH
)

In [ ]:
# Confirm custom dataset  registration
[
    data_set
    for data_set
    in MetadataCatalog.list()
    if data_set.startswith(DATA_SET_NAME)
]

In [ ]:
# Vizialize images from the dataset
metadata = MetadataCatalog.get(TRAIN_DATA_SET_NAME)
dataset_train = DatasetCatalog.get(TRAIN_DATA_SET_NAME)

dataset_entry = dataset_train[0]
image = cv2.imread(dataset_entry["file_name"])

visualizer = Visualizer(
    image[:, :, ::-1],
    metadata=metadata, 
    scale=0.8, 
    instance_mode=ColorMode.IMAGE_BW
)

out = visualizer.draw_dataset_dict(dataset_entry)
cv2_imshow(out.get_image()[:, :, ::-1])

## Train Model Using HERIDAL dataset 

### Configuration

In [ ]:
# HYPERPARAMETERS
ARCHITECTURE = "faster_rcnn_R_101_FPN_3x"
CONFIG_FILE_PATH = f"COCO-Detection/{ARCHITECTURE}.yaml"
MAX_ITER = 2000
EVAL_PERIOD = 200
BASE_LR = 0.001
NUM_CLASSES = 3

# OUTPUT DIR
OUTPUT_DIR_PATH = os.path.join(
    DATA_SET_NAME, 
    ARCHITECTURE, 
    datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
)


In [ ]:
os.makedirs(OUTPUT_DIR_PATH, exist_ok=True)

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(CONFIG_FILE_PATH))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(CONFIG_FILE_PATH)
cfg.DATASETS.TRAIN = (TRAIN_DATA_SET_NAME,)
cfg.DATASETS.TEST = (TRAIN_DATA_SET_NAME,) # Use train dataset for evaluation
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.TEST.EVAL_PERIOD = EVAL_PERIOD
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = BASE_LR
cfg.SOLVER.MAX_ITER = MAX_ITER
cfg.MODEL.ROI_HEADS.NUM_CLASSES = NUM_CLASSES
cfg.OUTPUT_DIR = OUTPUT_DIR_PATH


### Training

In [ ]:
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir $OUTPUT_DIR_PATH

### Evaluation

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
predictor = DefaultPredictor(cfg)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!ls

In [ ]:
# Vizualize predctions
dataset_valid = DatasetCatalog.get(TRAIN_DATA_SET_NAME)

for d in dataset_valid:
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)
    
    visualizer = Visualizer(
        img[:, :, ::-1],
        metadata=metadata, 
        scale=0.8, 
        instance_mode=ColorMode.IMAGE_BW
    )
    out = visualizer.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# Set the path to the saved model weights
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")

# Set the testing score threshold
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7

# Create the COCOEvaluator
evaluator = COCOEvaluator(TRAIN_DATA_SET_NAME, cfg, False, output_dir=cfg.OUTPUT_DIR)

# Build the test loader
test_loader = build_detection_test_loader(cfg, TRAIN_DATA_SET_NAME)

# Run the evaluation
metrics = inference_on_dataset(predictor.model, test_loader, evaluator)